### 

*Funtions*
- Max payout

Put ; Put Spread ; Put Fly ; Put Spread ; Collar
Strangle
Risk Reversal, Call Spread, Call Fly

In [ ]:
import pandas as pd

In [5]:
# Example data structure for option trading strategies
data = {
    "Strategy": ["Put", "Put Spread", "Put Ratio", "Put Fly", "Put Spread Collar", "Strangle", "Risk Reversal", "Call", "Call Spread", "Call Ratio", "Call Fly"],
    "Strike1": [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100],
    "Strike2": [None, 90, None, 95, 90, 110, 90, None, 110, None, 105],
    "Strike3": [None, None, None, 105, None, None, None, None, None, None, 115],
    "S0": [120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120],
    "Sigma": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
    "T": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    "RiskFreeRate": [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
    "NumPaths": [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]
}

option_strategies_df = pd.DataFrame(data)
option_strategies_df

,Strategy,Strike1,Strike2,Strike3,S0,Sigma,T,RiskFreeRate,NumPaths
0,Put,100,NaN,NaN,120,0.2,1,0.01,10000
1,Put Spread,100,90.0,NaN,120,0.2,1,0.01,10000
2,Put Ratio,100,NaN,NaN,120,0.2,1,0.01,10000
3,Put Fly,100,95.0,105.0,120,0.2,1,0.01,10000
4,Put Spread Collar,100,90.0,NaN,120,0.2,1,0.01,10000
5,Strangle,100,110.0,NaN,120,0.2,1,0.01,10000
6,Risk Reversal,100,90.0,NaN,120,0.2,1,0.01,10000
7,Call,100,NaN,NaN,120,0.2,1,0.01,10000
8,Call Spread,100,110.0,NaN,120,0.2,1,0.01,10000
9,Call Ratio,100,NaN,NaN,120,0.2,1,0.01,10000


In [ ]:
def max_payout(strategy, strike1, strike2=None, strike3=None, premium1=0, premium2=0, premium3=0, underlying_price=0):
    if strategy == 'Put':
        return max(strike1 - premium1, 0)
    elif strategy == 'Put Spread':
        return max(min(strike1 - strike2, strike1 - premium1 - premium2), 0)
    elif strategy == 'Put Fly':
        # Simplified version for equidistant strikes
        return max(min(strike2 - strike1, strike1 - 2*premium1 + premium2), 0)
    elif strategy == 'Put Spread Collar':
        # Very simplified, assuming you own the stock
        return max(underlying_price - strike1 + strike1 - strike2 - premium1 - premium2, 0)
    elif strategy == 'Strangle':
        # Simplified, only considering the put side
        return max(strike1 - premium1, 0)
    elif strategy == 'Risk Reversal':
        # Simplified, only considering the call side
        return float('inf')
    elif strategy == 'Call':
        return float('inf')
    elif strategy == 'Call Spread':
        return max(min(strike2 - strike1, strike2 - strike1 - premium1 - premium2), 0)
    elif strategy == 'Call Fly':
        # Simplified version for equidistant strikes
        return max(min(strike2 - strike1, strike1 - 2*premium1 + premium2), 0)
    else:
        return 'Strategy not recognized'

# Example usage:
print(max_payout('Put', 100, premium1=5))


In [9]:
def simulate_asset_paths(S0, risk_free_rate, sigma, T, num_paths):
    """
    Simulates asset price paths using the Geometric Brownian Motion model.
    - S0: Initial stock price
    - risk_free_rate: Annual risk-free interest rate
    - sigma: Volatility (standard deviation) of stock returns
    - T: Time to maturity (in years)
    - num_paths: Number of paths to simulate
    """
    dt = T / 100  # Divide time to maturity into 100 steps
    paths = np.zeros((101, num_paths))
    paths[0] = S0
    for t in range(1, 101):
        Z = np.random.standard_normal(num_paths)  # Random component
        paths[t] = paths[t - 1] * np.exp((risk_free_rate - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
    return paths

def max_payout_monte_carlo(strategy, strike1, strike2=None, strike3=None, S0=100, sigma=0.2, T=1, risk_free_rate=0.05, num_paths=10000):
    """
    Estimates the maximum payout for various option strategies using Monte Carlo simulation.
    - strategy: Option strategy name
    - strike1, strike2, strike3: Strike prices for the options
    - S0, sigma, T, risk_free_rate: Market parameters as in simulate_asset_paths
    - num_paths: Number of simulated paths
    Returns the estimated maximum payout for the selected strategy.
    """
    paths = simulate_asset_paths(S0, risk_free_rate, sigma, T, num_paths)
    final_prices = paths[-1]  # Prices at maturity
    payoffs = np.zeros(num_paths)

    # Calculate payoffs based on strategy
    if strategy == 'Put':
        payoffs = np.maximum(strike1 - final_prices, 0)
    elif strategy == 'Put Spread':
        payoffs = np.maximum(np.minimum(final_prices - strike1, strike1 - strike2), 0)
    elif strategy == 'Put Ratio':
        # Custom calculation based on ratio and strikes
        pass
    elif strategy == 'Put Fly':
        payoffs = np.maximum(np.minimum(final_prices - strike1, strike2 - strike1), 0)
    elif strategy == 'Put Spread Collar':
        # Custom calculation based on strikes and underlying asset price
        pass
    elif strategy == 'Strangle':
        payoffs = np.maximum(np.maximum(final_prices - strike1, strike2 - final_prices), 0)
    elif strategy == 'Risk Reversal':
        payoffs = np.maximum(final_prices - strike1, 0) - np.maximum(strike2 - final_prices, 0)
    elif strategy == 'Call':
        payoffs = np.maximum(final_prices - strike1, 0)
    elif strategy == 'Call Spread':
        payoffs = np.maximum(np.minimum(final_prices - strike1, strike2 - strike1), 0)
    elif strategy == 'Call Ratio':
        # Custom calculation based on ratio and strikes
        pass
    elif strategy == 'Call Fly':
        payoffs = np.maximum(np.minimum(final_prices - strike1, strike2 - strike1), 0)

    return np.mean(payoffs) * np.exp(-risk_free_rate * T)

# Example usage
print(max_payout_monte_carlo('Put', 100, S0=120, sigma=0.2, T=1, risk_free_rate=0.01))


1.8679856367395824
